# Implementation of neural network training

We will be using what we have learned to train a neural network.

In particular, we will be using the adam update rule, considering numerical stability, and saving best set of parameters.

## Load the dataset

We will be using the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris).

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
data_path = os.path.join('data', 'iris.data')
dataset = pd.read_csv(data_path, names=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'class'])

In [3]:
len(dataset)  # Size of dataset

150

In [4]:
dataset[49:51]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
49,5.0,3.3,1.4,0.2,Iris-setosa
50,7.0,3.2,4.7,1.4,Iris-versicolor


The "class" column is the type of flower.

The other four column are the length and width of sepal and petal, these are the four features given in the dataset.

This time around we will train on the full data and try to classify all three classes.

### Input
The input will be vector of 4 scalars, the length and width of sepal and petal.

In [5]:
data = {}
fields = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
data['input'] = dataset.loc[:, fields].to_numpy(dtype=np.float_)

In [6]:
# A sample of input
data['input'][49:51]

array([[5. , 3.3, 1.4, 0.2],
       [7. , 3.2, 4.7, 1.4]])

### Target
The target will be a one-hot vector representing the class.

In [7]:
to_class = {0: "Iris-setosa", 1: "Iris-versicolor", 2: "Iris-virginica"}
to_index = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}

def to_one_hot(num_class, indexes):
    """Outputs a matrix of one hot vectors."""
    # Create vectors of zeros.
    n = len(indexes)
    one_hot = np.zeros((n, num_class), dtype=np.float_)
    # Set correct class to 1.
    one_hot[np.arange(n), indexes] = 1
    return one_hot

In [8]:
data['target'] = to_one_hot(3, [to_index[t] for t in dataset.loc[:,'class']])

In [9]:
# A sample of target
data['target'][49:51]

array([[1., 0., 0.],
       [0., 1., 0.]])

### Splitting and shuffling the data

We need to split the dataset into training data, validation data, and test data.

Since there is only 150 samples in the dataset, it is important to ensure there is a even distribution of classes in each type of data.

In [10]:
valid_ratio = 1/5
test_ratio = 1/5
# rest for training data

num_per_class = 50
v_n = int(num_per_class * valid_ratio)  # Number of valid per class
t_n = int(num_per_class * test_ratio)

valid_idx = np.r_[0:v_n, 50:50 + v_n, 100:100 + v_n]
test_idx = np.r_[v_n:v_n + t_n, 50 + v_n:50 + v_n + t_n, 100 + v_n:100 + v_n + t_n]
training_idx = np.r_[v_n:v_n + t_n, 50 + v_n:50 + v_n + t_n, 100 + v_n:100 + v_n + t_n]

valid = {}
test = {}
training = {}

valid['input'] = data['input'][valid_idx]
valid['target'] = data['target'][valid_idx]
test['input'] = data['input'][test_idx]
test['target'] = data['target'][test_idx]
training['input'] = data['input'][training_idx]
training['target'] = data['target'][training_idx]

Shuffle the training data

In [11]:
from sklearn.utils import shuffle

In [12]:
training['input'], training['target'] = shuffle(training['input'], training['target'])

### Loss function
We will use cross entropy loss as loss function. The softmax function and loss function is modified for numerical stability as explained in the previous lesson.

The cross entropy loss is modified to prevent calculating the log of 0, which is infinity and will overflow. 

In [13]:
def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)

class CrossEntropyLoss:
    @staticmethod
    def forward(x, t):
        x = softmax(x)
        x = np.maximum(1e-8, x)  # Prevent x = 0
        return -np.log(x) @ t
        
    @staticmethod
    def backward(x, t):
        x = softmax(x)
        return x - t

### Activation function

We will use ReLU as activation function.

In [14]:
class ReLU:
    
    @staticmethod
    def forward(x):
        return np.maximum(0, x)
    
    @staticmethod
    def backward(x):
        x[x <= 0] = 0
        x[x > 0] = 1
        return x

### Update rule

We will be using the Adam optimizer. The implementation is modified to take Python dictionary of params and update them.

In [15]:
class Adam:
    
    def __init__(self, alpha, beta1, beta2, epsilon):
        self.a = alpha
        self.b1 = beta1
        self.b2 = beta2
        self.eps = epsilon
        self.t = 0
    
    def set_param(self, params):
        """Create the storage for m and n values for a given parameter set."""
        self.m = {}
        self.n = {}
        for key, lst in params.items():
            m_lst = []
            n_lst = []
            for param in lst:
                m_lst.append(np.zeros_like(param))
                n_lst.append(np.zeros_like(param))
            self.m[key] = m_lst
            self.n[key] = n_lst
    
    def __call__(self, params, grads):
        self.t += 1
        for key in params:
            num = len(params[key])
            for i in range(num):
                self.m[key][i] = self.b1 * self.m[key][i] + (1 - self.b1) * grads[key][i]
                self.n[key][i] = self.b1 * self.n[key][i] + (1 - self.b1) * (grads[key][i] ** 2)
                m_hat = self.m[key][i] / (1 - self.b1 ** self.t)
                n_hat = self.n[key][i] / (1 - self.b2 ** self.t)
                params[key][i] -= self.a * m_hat / (np.sqrt(n_hat) + self.eps)
        return

### Modular network implementation

The backpropagation implementation in previous lesson only works for networks which solely consist of fully connected feed forward layers with same activation function.

Many common network include multiple type of layers and different types of activation function.

The common frameworks to train neural network offers a module approach.

Rather than using the delta short hand, we will be using the following formula:

\begin{align}
& \frac{\partial E}{\partial w_{ij}} = \frac{\partial E}{\partial o_j} \cdot \frac{\partial h_j(s_j)}{\partial s_j} \cdot o_i \\
& \frac{\partial E}{\partial o_j} = \begin{cases} \frac{\partial E(t_j, o_j)}{\partial o_j} & \text{if $j$ is an output neuron} \\ \sum_{l \in L} \frac{\partial E}{\partial o_l} \cdot \frac{\partial h_l(s_l)}{\partial s_l} \cdot w_{jl} & \text{otherwise}\end{cases}
\end{align}

Implementation of the fully connected feed forward layer:

In [16]:
class FullyConnected:
    
    def __init__(self, input_size, output_size, activation):
        self.weight = np.random.rand(input_size, output_size)
        self.bias = np.zeros(output_size)
        self.activation = activation
        self.x_cache = None
    
    def register_param(self):
        """Return all tunable parameters of this layer"""
        return [self.weight, self.bias]
        
    def forward(self, x):
        """Calculate layer output with given input."""
        # Store network input in cache.
        self.x_cache = x
       
        x = (x @ self.weight) + self.bias
        x = self.activation.forward(x)
        return x
    
    def backward(self, output_der):
        """Calculate derivative of layer weights and biases with given layer output derivative"""
        s_j_der = self.activation.backward(self.x_cache @ self.weight)
        weight_grad = np.outer(self.x_cache, output_der * s_j_der)
        bias_grad = output_der * s_j_der
        return [weight_grad, bias_grad]
    
    def input_der(self, output_der):
        """Helper function to calculate layer input derivative to be used for the previous layer"""
        s_j_der = self.activation.backward(self.x_cache @ self.weight)
        return self.weight @ (output_der * s_j_der)

The `forward` function takes the layer inputs in vector form and produce layer outputs in vector form.

The `backward` function takes the partial derivative of the loss with respect to the layer output ($\frac{\partial E}{\partial o_j}$ of the whole layer) and produce the partial derivative of the loss with respect to the layer weight ($\frac{\partial E}{\partial w_{ij}}$ of whole layer)

The `input_der` function takes the partial derivative of the loss with respect to the layer output ($\frac{\partial E}{\partial o_j}$ of the whole layer) and produce the partial derivative of the loss with respect to the layer input. This is used as the partial derivative of the loss with respect to the layer output for the previous layer ($\frac{\partial E}{\partial o_j}$ of the whole layer).

### Network structure

The input layer has 4 units, since the input is 4 scalars.

There are 2 hidden layers of 100 units.

The output layer has 3 units, since the output is 3 classes.

The activation function is ReLU for all layers.

In [17]:
class IrisNetwork:
    
    def __init__(self, loss, optimizer):
        self.loss = loss
        self.optim = optimizer
        self.layers = [FullyConnected(4, 100, ReLU), 
                       FullyConnected(100, 100, ReLU),
                       FullyConnected(100, 3, ReLU)]
        
        # Register parameters from each layer
        self.params = {}
        for i, layer in enumerate(self.layers):
            self.params[i] = layer.register_param()
        self.optim.set_param(self.params)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, x, t):
        self.grads = {}
        
        output_der = self.loss.backward(x, t)
        for i, layer in reversed(list(enumerate(self.layers))):
            der_param = layer.backward(output_der)
            self.grads[i] = der_param
            # Calculate output derivative for previous layer.
            output_der = layer.input_der(output_der)
        return
    
    def update(self):
        self.optim(self.params, self.grads)

### Training

Train the model, and save best model on validation data during training.

In [18]:
adam = Adam(0.01, 0.9, 0.999, 1e-8)
loss = CrossEntropyLoss

# Initialize the network
iris_network = IrisNetwork(loss, adam)

epoch = 50  # How many time to train the network with the whole training data.
num_e = 5

In [19]:
valid_accu = 0
training_loss = 0

for e in range(epoch):     
    # Test with validation data
    correct_count = 0
    total_count = 0
    for inputs, targets in zip(valid['input'], valid['target']):
        outputs = iris_network.forward(inputs)

        total_count += 1
        if np.argmax(outputs) == np.argmax(targets):
            correct_count += 1

    accu = correct_count / total_count
    
    # Save model if current best
    if accu >= valid_accu:
        valid_accu = accu
        best_params = iris_network.params
    
    # Train network with training data
    for inputs, targets in zip(training['input'], training['target']):
        outputs = iris_network.forward(inputs)
        iris_network.backward(outputs, targets)
        iris_network.update()
        
        training_loss += loss.forward(outputs, targets) / num_e

    # Print training info every set amount of epoch
    if not (e + 1) % num_e:
        print(f"At {e + 1: 5} epoch,\tvalidation accuracy: {valid_accu * 100:.2f}\tLoss: {training_loss}")
        training_loss = 0
        
# Load network with best parameters
iris_network.params = best_params

At     5 epoch,	validation accuracy: 66.67	Loss: 327.7365285997662
At    10 epoch,	validation accuracy: 66.67	Loss: 293.23129308878424
At    15 epoch,	validation accuracy: 66.67	Loss: 245.31443423714842
At    20 epoch,	validation accuracy: 100.00	Loss: 170.7149215299162
At    25 epoch,	validation accuracy: 100.00	Loss: 109.98302591645019
At    30 epoch,	validation accuracy: 100.00	Loss: 51.77957669560662
At    35 epoch,	validation accuracy: 100.00	Loss: 58.97381317610746
At    40 epoch,	validation accuracy: 100.00	Loss: 6.015790340378242
At    45 epoch,	validation accuracy: 100.00	Loss: 15.82838167519875
At    50 epoch,	validation accuracy: 100.00	Loss: 5.216048953594284


### Test the network

In [20]:
# Test the network:
correct_count = 0
total_count = 0
for inputs, targets in zip(test['input'], test['target']):
    outputs = iris_network.forward(inputs)

    total_count += 1
    if np.argmax(outputs) == np.argmax(targets):
        correct_count += 1

accu = correct_count / total_count

print(f"Network accuracy: {accu * 100}%")

Network accuracy: 100.0%


Normally the difference between the validation accuracy and test accuracy should be small, but since we are using a small dataset, it might be larger.

Either way, the network should be trained faster and performing better on a harder task (classification of three classes compared to two classes).

## Tools and library for training networks

The two most common python libraries for training neural network are: PyTorch and TensorFlow.

Both framework work similar to our modular implementation. However, nowadays both framework supports automatic differentiation, which means you don't have to implement the derivative functions if using predefined modules by the framework (or if your module is solely consist of predefined modules).

Usually your module is going to be solely consist of predefined modules, since basic operations such as addition, multiplication, etc. are already predefined. Therefore to use these frameworks, you only need to define the structure of the network, loss function, learning rate, and optimizer/update rule. The rest will be taken care of by the framework.

However, it is still important to understand how the framework works. This knowledge can help in debugging, and improving models. Especially, if you are developing a new module.

## Practice

### Exercise 1

Add gradient clipping to the above neural network implementation.

In [21]:
# code here

### Exercise 2

Train the network with gradient clipping. Find a good set of hyperparameters (learning rate, etc.).

In [22]:
# code here

## Solution

In [23]:
# Solution to exercise 1.

def gradient_clipping(g, T):
    g_norm = np.linalg.norm(g)
    if g_norm > T:
        g = T * g / g_norm
    return g

class GradClipNetwork:
    
    def __init__(self, loss, optimizer, t):
        self.loss = loss
        self.optim = optimizer
        self.t = t
        self.layers = [FullyConnected(4, 100, ReLU), 
                       FullyConnected(100, 100, ReLU),
                       FullyConnected(100, 3, ReLU)]
        
        # Register parameters from each layer
        self.params = {}
        for i, layer in enumerate(self.layers):
            self.params[i] = layer.register_param()
        self.optim.set_param(self.params)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, x, t):
        self.grads = {}
        
        output_der = self.loss.backward(x, t)
        for i, layer in reversed(list(enumerate(self.layers))):
            der_param = layer.backward(output_der)
            self.grads[i] = der_param
            # Calculate output derivative for previous layer.
            output_der = layer.input_der(output_der)
        return
    
    def update(self):
        # Gradient clipping
        for key in self.grads:
            n = len(self.grads[key])
            for i in range(n):
                self.grads[key][i] =  gradient_clipping(self.grads[key][i], self.t)
        self.optim(self.params, self.grads)

In [24]:
# Solution to exercise 2

adam = Adam(0.05, 0.9, 0.999, 1e-8)
loss = CrossEntropyLoss

# Initialize the network
grad_clip_network = GradClipNetwork(loss, adam, 1)

epoch = 50
num_e = 5


valid_accu = 0
training_loss = 0

for e in range(epoch):     
    # Test with validation data
    correct_count = 0
    total_count = 0
    for inputs, targets in zip(valid['input'], valid['target']):
        outputs = grad_clip_network.forward(inputs)

        total_count += 1
        if np.argmax(outputs) == np.argmax(targets):
            correct_count += 1

    accu = correct_count / total_count
    
    # Save model if current best
    if accu >= valid_accu:
        valid_accu = accu
        best_params = grad_clip_network.params
    
    # Train network with training data
    for inputs, targets in zip(training['input'], training['target']):
        outputs = grad_clip_network.forward(inputs)
        grad_clip_network.backward(outputs, targets)
        grad_clip_network.update()
        
        training_loss += loss.forward(outputs, targets) / num_e

    # Print training info every set amount of epoch
    if not (e + 1) % num_e:
        print(f"At {e + 1: 5} epoch,\tvalidation accuracy: {valid_accu * 100:.2f}\tLoss: {training_loss}")
        training_loss = 0
        
# Load network with best parameters
grad_clip_network.params = best_params

At     5 epoch,	validation accuracy: 76.67	Loss: 246.1727970135372
At    10 epoch,	validation accuracy: 100.00	Loss: 27.233251985620285
At    15 epoch,	validation accuracy: 100.00	Loss: 29.36586529646044
At    20 epoch,	validation accuracy: 100.00	Loss: 27.27442856229027
At    25 epoch,	validation accuracy: 100.00	Loss: 27.13470307505745
At    30 epoch,	validation accuracy: 100.00	Loss: 14.491447500240628
At    35 epoch,	validation accuracy: 100.00	Loss: 12.543871939348575
At    40 epoch,	validation accuracy: 100.00	Loss: 14.727432621161494
At    45 epoch,	validation accuracy: 100.00	Loss: 15.337508327065628
At    50 epoch,	validation accuracy: 100.00	Loss: 7.441114048989004
